# Solving a problem with quantum computers

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer
from qiskit.visualization import plot_histogram, array_to_latex
from qiskit.quantum_info import Statevector
import numpy as np
backend = BasicAer.get_backend('qasm_simulator')

Generating random numbers from a variety of specific probability distributions is interesting, and so is implementing digital computer operations on a quantum computer so that multiple operations are performed simulaneously. However neither of these is enough to justify the hype around quantum computers. Let's now take a look at an example of something where quantum computers can significantly outperform digital computers. It's known as [Grover's algorithm](https://en.wikipedia.org/wiki/Grover%27s_algorithm), and allows a quantum computer to take a function that might be performed on digital computers, and wrap it in some quantum goodness that quickly solves for it. Specifically, the function is a type where it has one (or just a couple) of solutions, and where the approach to solve for it on a digital computer would be to "brute force" the answer by trying every possible solution to check if it works. On a quantum computer, there are some tricks to try the function fewer times, or even just once, and yet still figure out the solution.

Functions of this type, that have one specific solution, exist all over the place. So, this is a potentially highly useful application for quantum computers. For example, a function that checks a possible set of numbers to a particular [Sudoku puzzle](https://en.wikipedia.org/wiki/Sudoku) to confirm if it is a correct solution, a function that checks a password to see if it matches an encrypted password entry for a user, or a function that confirms the colour of a particular pixel in an image is correct for a given 3D scene with a particular set of objects and lighting. Many different problems can be rewritten in terms of a function that checks if an answer is correct.

However, before seeing how to do this on a quantum computer, we need to introduce a couple of new operations. 

https://en.wikipedia.org/wiki/Grover%27s_algorithm
https://www.ictp-saifr.org/wp-content/uploads/2022/11/ICTP_SAIFR_D1-L2.pdf
https://qiskit.org/ecosystem/algorithms/tutorials/06_grover.html
https://learn.qiskit.org/course/ch-algorithms/grovers-algorithm
https://www.quantum-inspire.com/kbase/grover-algorithm/

### Z operation
The Z operation works on all pairs of rows in the state vector associated with different values of a particular qubit, and flips the sign on the second row of each pair. We can call it the "flip" operation. Let's have a quick look at an example.

If we start with an H(0) operator and an H(1) operator, as we did in the first notebook, we have the same value on each row of the state vector. If we then do a Z(0) follwed by a Z(1), you can see the signs flip but the numbers otherwise stay the same.
| Qubits| Initial state vector | H(0) | H(1) | Z(0) | Z(1) |
| :- | :- | :- | :- | :- | :- |
| $|00\rangle$ | $1.0$ | $\frac{1}{\sqrt{2}}$ | $\frac{1}{2}$ | $\frac{1}{2}$  | $\frac{1}{2}$  |
| $|01\rangle$ | $0.0$ | $\frac{1}{\sqrt{2}}$ | $\frac{1}{2}$ | -$\frac{1}{2}$ | -$\frac{1}{2}$ |
| $|10\rangle$ | $0.0$ | $0.0$                | $\frac{1}{2}$ | $\frac{1}{2}$  | -$\frac{1}{2}$ |
| $|11\rangle$ | $0.0$ | $0.0$                | $\frac{1}{2}$ | -$\frac{1}{2}$ | $\frac{1}{2}$  |

We saw negative probabilities in the notebook where we introduced the RY operation, and here they are again. They are the key to how Grover's algorithm works.

### CCZ operation
Similarly to CCX, the CCZ is "doubly constrained". In this case, it is a "doubly constrained flip" operation. Constrained to just those rows where the two specified qubits are $|1\rangle$, it flips the sign of the second row of all pairs where the third qubit is the only one changing. Since the second row of these pairs is also the row where the third qubit is $|1\rangle$, another way to think about this operation is flipping the sign of all rows where the three specified qubits are $|1\rangle$.

Here's an example of CCZ in practice:
| Qubits| Initial state vector | H(0) | H(1) | H(2) | CCZ(0, 1, 2) |
| :- | :- | :- | :- | :- | :- |
| $|000\rangle$ ($|0\rangle$) | $1.0$ | $\frac{1}{\sqrt{2}}$ | $\frac{1}{2}$ | $\frac{1}{\sqrt{8}}$ | $\frac{1}{\sqrt{8}}$ |
| $|001\rangle$ ($|1\rangle$) | $0.0$ | $\frac{1}{\sqrt{2}}$ | $\frac{1}{2}$ | $\frac{1}{\sqrt{8}}$ | $\frac{1}{\sqrt{8}}$ |
| $|010\rangle$ ($|2\rangle$) | $0.0$ | $0.0$ | $\frac{1}{2}$ | $\frac{1}{\sqrt{8}}$ | $\frac{1}{\sqrt{8}}$ |
| $|011\rangle$ ($|3\rangle$) | $0.0$ | $0.0$ | $\frac{1}{2}$ | $\frac{1}{\sqrt{8}}$ | $\frac{1}{\sqrt{8}}$ |
| $|100\rangle$ ($|4\rangle$) | $0.0$ | $0.0$ | $0.0$ | $\frac{1}{\sqrt{8}}$ | $\frac{1}{\sqrt{8}}$ |
| $|101\rangle$ ($|5\rangle$) | $0.0$ | $0.0$ | $0.0$ | $\frac{1}{\sqrt{8}}$ | $\frac{1}{\sqrt{8}}$ |
| $|110\rangle$ ($|6\rangle$) | $0.0$ | $0.0$ | $0.0$ | $\frac{1}{\sqrt{8}}$ | $\frac{1}{\sqrt{8}}$ |
| $|111\rangle$ ($|7\rangle$) | $0.0$ | $0.0$ | $0.0$ | $\frac{1}{\sqrt{8}}$ | $-\frac{1}{\sqrt{8}}$ |


### Implementing a verifier
The other thing that Grover's algorithm needs is a function that verifies whether a value is a valid solution to some problem. All it needs to do is take a potential solution, and tell us "yes" or "no".

We can do this by considering some of the qubits to represent a proposed solution, and one other qubit to represent "yes" if it is $|1\rangle$ or "no" if it is $|0\rangle$. In the example implemented here, qubits 0 and 1 will represent potential solutions, and qubit 2 will represent the result of validating it.

On a digital computer, we would think about this as bits. We would implement some logical operations that take two bits representing potential solutions and return another bit with the validation result. As we saw in the last notebook, we can implement the deterministic operations of a digital computer on a quantum computer by using X, CX, CCX, etc. operations.

Let's say we want our verifier function to take state vectors where one of $|000\rangle$, $|001\rangle$, $|010\rangle$, or $|011\rangle$ rows has the value $1.0$ (100%), and only if it's the "right" one, will the state vector be changed so that the corresponding row where qubit 2 is $|1\rangle$ becomes $1.0$. For example, if $|011\rangle$ is the right solution, this would be implemented simply with the function CCX(0, 1, 2).

Firstly, let's use X operations to encode the value 3 into the state vector, by putting the value $1.0$ in the $|011\rangle$ ($|3\rangle$) row.

In [9]:
q = QuantumRegister(3) # We want 3 qubits
c = ClassicalRegister(2) # but we care about only 2 bits
circuit = QuantumCircuit(q, c) # Sets up the quantum computer, a.k.a. quantum circuit
circuit.x(0)
circuit.x(1)

v1 = Statevector(circuit)
array_to_latex(np.vstack(v1.data))

<IPython.core.display.Latex object>

Now if we perform CCX(0, 1, 2), the values in $|011\rangle$ ($|3\rangle$) and $|111\rangle$ ($|7\rangle$) will be swapped, moving the $1.0$ value to the final row, where qubit 2 has a value of $|1\rangle$. Since we know that CCX is constrained to work only on these rows, we know that only where the $|011\rangle$ potential solution is given the $1.0$ value will the state vector be changed to have $1.0$ on a row where qubit 2 is $|1\rangle$. The other three potential solutions will result in no change.

In [10]:
circuit.ccx(0, 1, 2)
v2 = Statevector(circuit)
array_to_latex(np.vstack(v2.data))

<IPython.core.display.Latex object>

To create different verifier functions, we can use X operations and specify either qubit 0 or qubit 1. For example, to create a function that will answer "yes" for the potential solution $|01\rangle$ and "no" the other three potential solutions, we simply do X(1) before doing CCX(0, 1, 2). We will also do X(1) again after the CCX to "undo" the first X, and ensure the state vector has $1.0$ in the $|101\rangle$ ($|5\rangle$) row:
| Qubits| Initial state vector | X(1) | CCX(0, 1, 2) | X(1) |
| :- | :- | :- | :- | :- |
| $|000\rangle$ ($|0\rangle$) | $0.0$ | $0.0$ | $0.0$ | $0.0$ |
| $|001\rangle$ ($|1\rangle$) | $1.0$ | $0.0$ | $0.0$ | $0.0$ |
| $|010\rangle$ ($|2\rangle$) | $0.0$ | $0.0$ | $0.0$ | $0.0$ |
| $|011\rangle$ ($|3\rangle$) | $0.0$ | $1.0$ | $0.0$ | $0.0$ |
| $|100\rangle$ ($|4\rangle$) | $0.0$ | $0.0$ | $0.0$ | $0.0$ |
| $|101\rangle$ ($|5\rangle$) | $0.0$ | $0.0$ | $0.0$ | $1.0$ |
| $|110\rangle$ ($|6\rangle$) | $0.0$ | $0.0$ | $0.0$ | $0.0$ |
| $|111\rangle$ ($|7\rangle$) | $0.0$ | $0.0$ | $1.0$ | $0.0$ |

Implementing this in Qiskit:

In [4]:
# Verifies that a proposed solution is correct only when it is |10>
def add_verify(circuit):
    circuit.x(1)
    circuit.ccx(0, 1, 2)
    circuit.x(1)

In [5]:
circuit = QuantumCircuit(q, c) # Sets up the quantum computer, a.k.a. quantum circuit
circuit.x(0) # Ensure the state vector has 100% in the |001> row
add_verify(circuit)
v3 = Statevector(circuit)
array_to_latex(np.vstack(v3.data))

<IPython.core.display.Latex object>

However, we need to modify the verifier function a little first. We are going to apply the H operation on the result qubit (qubit 2) before running the function, and then again afterwards.

This is a little trick that turns an X operation into a Z operation. So, the CCX operation effectively becomes like a CCZ operation. And yes, the verifier function could have just been written with a CCZ instead of a CCX and we could skip the H operations, but digital computer operations don't use Z type operations, and this way the algorithm is more general.

In [7]:
def add_verify_with_h(circuit):
    circuit.h(2)
    add_verify(circuit)
    circuit.h(2)

This version of the function will now flip the sign of the state vector row with the answer. Unfortunately, we can't stop here with the job done, because in practice we can't read the state vector out of the quantum computer. All we can do is take measurements of the qubits, and while we can have a negative value in a row of a state vector, we won't see a negative probabilty appear in measurements.

Grover's algorithm is about amplifying the negative row so it will have a higher probability in the measurements.

### Grover's algorithm
Normally the verification function will be quite complicated, and difficult to figure out from just looking at it. Our verification function is simple, but that's fine for learning how Grover's algorithm works.

The basic strategy for using Grover's algorithm is to:
1. Prepare the state vector so it has the same value on every row, i.e. no row has a zero value.
2. Apply the verification function, which will flip the sign of the row corresponding to the right answer.
3. Amplify the negative rows compared to the non-negative rows.

Then we measure the qubits, and the most likely result should be the right answer. For larger numbers of qubits, the steps 2 and 3 will typically be repeated to make the right answer clearer, but we shouldn't need to do that for our example.

We've already defined the verification function, but here's the state preparation function:

In [8]:
def add_prepare(circuit):
    circuit.h(0)
    circuit.h(1)
    circuit.h(2)

It is just the approach to creating a uniform probability distribution that we saw in the first notebook.

We can see how the verification function just flips the sign of the answer row ($|101\rangle$) when given a state vector with $\frac{1}{\sqrt{8}}$ values in all of its rows:

In [19]:
circuit = QuantumCircuit(q, c) # Sets up the quantum computer, a.k.a. quantum circuit
add_prepare(circuit)
add_verify_with_h(circuit)
v4 = Statevector(circuit)
array_to_latex(np.vstack(v4.data))

<IPython.core.display.Latex object>

In [21]:
circuit.h(1)
v = Statevector(circuit)
array_to_latex(np.vstack(v.data))

<IPython.core.display.Latex object>

The amplification function requires a bit of explanation.

In [13]:
add_prepare(circuit)
v5 = Statevector(circuit)
array_to_latex(np.vstack(v5.data))

<IPython.core.display.Latex object>

In [ ]:

def add_amplify(circuit):
    prepare(circuit)
    circuit.x(0)
    circuit.x(1)
    circuit.x(2)
    circuit.ccz(0, 1, 2)
    circuit.x(0)
    circuit.x(1)
    circuit.x(2)
    prepare(circuit)
    
q = QuantumRegister(3) # We want 3 qubits
c = ClassicalRegister(2) # but want to measure only 2 of them, so we need 2 bits
circuit = QuantumCircuit(q, c) # Sets up the quantum computer, a.k.a. quantum circuit

add_prepare(circuit)
add_flipping_verify(circuit)
add_amplify(circuit)

circuit.measure(q[0:2], c)          # Measure the two qubits 0 and 1, and get some bits
result = execute(circuit, backend, shots=1000).result()    # Run this all 1,000 times
plot_histogram(result.get_counts(circuit))                 # Show a histogram of the results

### Grover's algorithm
Let's put these new operations to work.

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer
from qiskit.visualization import plot_histogram, array_to_latex
from qiskit.quantum_info import Statevector
import numpy as np
backend = BasicAer.get_backend('qasm_simulator')

def add_cry(c, theta, x, y):
    c.ry(theta/2, y)
    c.cx(x, y)
    c.ry(-theta/2, y)
    c.cx(x, y)

def add_ccx(c, x, y, z):
    add_cry(c, np.pi/2, y, z)
    c.cx(x, y)
    add_cry(c, -np.pi/2, y, z)
    c.cx(x, y)
    add_cry(c, np.pi/2, x, z)

def add_ccz(c, x, y, z):
    c.h(z)
    add_ccx(c, x, y, z)
    c.h(z)

q = QuantumRegister(3) # We want 3 qubits
c = ClassicalRegister(2) # and also 2 bits
circuit = QuantumCircuit(q, c) # Sets up the quantum computer, a.k.a. quantum circuit

circuit.h(0)
circuit.h(1)
#circuit.x(2) # Need to include this if use X rather than RY operations
circuit.h(2)

# Black box that does X on qubit 2 iff qubits 0 and 1 are in the secret state (and leaves/returns them in that state).
# Secret state is |01>, i.e. qubit 0 = |1>, qubit 1 = |0>.
circuit.ry(np.pi, 1)
add_ccx(circuit, 0, 1, 2)
circuit.ry(-np.pi, 1)
#This one works - equivalent to flipping the sign on the |001> row if everything is in equal superposition.
#Also, only swaps to a state in the second half of the state vector if start in |001> row
#circuit.x(1)
#circuit.ccx(0,1,2)
#circuit.x(1)
v4 = Statevector(circuit)

circuit.h(0)
circuit.h(1)
circuit.h(2)

# Flip the sign on the |000> row:
# Next three lines are equivalent to reversing the state vector
circuit.ry(np.pi,0)
circuit.ry(np.pi,1)
circuit.ry(np.pi,2)
#add_ccz(circuit, 0, 1, 2)
circuit.ccz(2, 1, 0)
# Then we "un-reverse" the state vector
circuit.ry(-np.pi,0)
circuit.ry(-np.pi,1)
circuit.ry(-np.pi,2)

circuit.h(0)
circuit.h(1)
circuit.h(2)

array_to_latex(np.vstack(v4.data))

#circuit.measure(q[0:2], c)          # Measure the two qubits 0 and 1, and get some bits
#result = execute(circuit, backend, shots=1000).result()    # Run this all 1,000 times
#plot_histogram(result.get_counts(circuit))                 # Show a histogram of the results

In [ ]:
q = QuantumRegister(3) # We want 3 qubits
c = ClassicalRegister(3) # and also 3 bits
circuit = QuantumCircuit(q, c) # Sets up the quantum computer, a.k.a. quantum circuit

circuit.h(0)
circuit.h(1)
circuit.x(2)
circuit.h(2)
verifier(circuit)

circuit.h(0)
circuit.h(1)
circuit.h(2)

#circuit.ry(np.pi,0)
#circuit.ry(np.pi,1)
#circuit.ry(np.pi,2)
circuit.x(0)
circuit.x(1)
circuit.x(2)
#add_ccz(circuit, 0, 1, 2)
circuit.ccz(2, 1, 0)
# Then we "un-reverse" the state vector
#circuit.ry(-np.pi,0)
#circuit.ry(-np.pi,1)
#circuit.ry(-np.pi,2)
circuit.x(0)
circuit.x(1)
circuit.x(2)

v4 = Statevector(circuit)
array_to_latex(np.vstack(v4.data))